https://www.youtube.com/watch?v=oWZQb8nRGfk&ab_channel=ArturSpirin

In [1]:
class Table:
    
    def __init__(self, driver):
        self.driver = driver
        
    def get_column_info(self):
        
        column_info = []
        columns = self.driver.find_elements_by_xpath('/html/body/div[2]/div[3]/div/table[2]/thead/tr/th')
        for column in columns:
            column_info.append(str(column.text))
        return column_info
        
    def get_results(self, index=None):
        columns = self.get_column_info()
        data = {}
        elements = self.driver.find_elements_by_xpath('/html/body/div[2]/div[3]/div/table[2]/tbody/tr{}'
                                                      .format('[{}]'.format(index) if index else ''))
        
        for element in elements:
            current_index = elements.index(element) + 1 if not index else index #xpath counter starts at 1
            parsed_data = {}
            for column in columns:
                value = element.find_element_by_xpath('/html/body/div[2]/div[3]/div/table[2]/tbody/tr[{}]/td[{}]'
                                                      .format(current_index, columns.index(column)+1)).text
                parsed_data.update({column: str(value)})
            
            data.update({current_index: parsed_data})
            
        return data
    
    def get_number_of_results(self):
        
        return len(self.driver.find_elements_by_xpath('/html/body/div[2]/div[3]/div/table[2]/tbody/tr'))
    
    def click(self, column, value):
        
        pass

In [2]:
import pandas as pd
import numpy as np

from selenium import webdriver
import time

In [3]:
start_year = 2000
end_year = 2021

driver = webdriver.Chrome(executable_path='C:/Users/17178/Downloads/chromedriver_win32/chromedriver.exe')

for year in range(start_year, end_year):
    driver.get('https://basketball.realgm.com/nba/draft/past_drafts/{}'.format(str(year)))

    table = Table(driver)

    if year == start_year:
        df = pd.DataFrame(table.get_results()).T
        df['draft_year'] = year
        df = df.reset_index(drop=True)
    else:
        df_temp = pd.DataFrame(table.get_results()).T
        df_temp['draft_year'] = year
        df = pd.concat([df, df_temp])
        df = df.reset_index(drop=True)
df
#     print(table.get_column_info())
#     print(table.get_results())
#     print(table.get_number_of_results())

,Pick,Player,Team,Draft Trades,Pos,HT,WT,Age,YOS,Pre-Draft Team,Class,Nationality,draft_year,
0,30,Marko Jaric,LAC,,GF,6-7,224,21,8,Fortituto Kontatto Bologna (Italy),1978 DOB,Serbia / Greece,2000,NaN
1,31,Dan Langhi,DAL,DAL to HOU,FC,6-11,220,22,4,Vanderbilt,Sr,United States,2000,NaN
2,32,A.J. Guyton,CHI,,PG,6-1,180,22,4,Indiana,Sr,United States,2000,NaN
3,33,Jake Voskuhl,CHI,,C,6-11,245,22,9,Connecticut,Sr,United States,2000,NaN
4,34,Khalid El-Amin,CHI,,G,5-10,200,21,3,Connecticut,Jr *,United States,2000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,56,Grant Riller,CHA,,SG,6-3,190,23,1,Charleston,Sr,United States,2020,NaN
622,57,Reggie Perry,LAC,LAC to BRK,F,6-8,250,20,1,Mississippi State,So *,United States,2020,NaN
623,58,"Paul Reed, Jr.",PHL,,F,6-8,218,21,1,DePaul,Jr *,United States,2020,NaN
624,59,Jalen Harris,TOR,,SG,6-5,195,22,1,Nevada,Jr *,United States,2020,NaN


In [4]:
df.to_csv('C:/Users/17178/Documents/DS Projects/FanDuel/NBA Draft Project/Data/nba_draft_second_round.csv')